![Banner](https://i.imgur.com/a3uAqnb.png)

# CycleGAN for Selfie-to-Anime Translation - Homework Assignment

In this homework, you will implement a **CycleGAN** for unpaired image-to-image translation between selfie and anime domains. CycleGANs enable learning mappings between two domains without requiring paired training examples.

## 📌 Project Overview
- **Task**: Unpaired image-to-image translation (Selfie ↔ Anime)
- **Dataset**: Selfie2Anime dataset from Kaggle
- **Architecture**: CycleGAN with Generator and Discriminator networks
- **Goal**: Generate realistic anime-style images from selfies and vice versa

## 📚 Learning Objectives
By completing this assignment, you will:
- Understand adversarial training and GAN architectures
- Implement cycle consistency and identity loss functions
- Build Generator and Discriminator networks
- Train unpaired image translation models
- Evaluate generative model performance

![CycleGAN Process](https://www.oreilly.com/api/v2/epubs/9781788836067/files/assets/d5036aa6-77cf-41c1-8828-11436977198e.png)

[Image Source](https://www.oreilly.com/library/view/hands-on-artificial-intelligence/9781788836067/c2e7d914-4e45-4528-8627-c590d19107ef.xhtml)

## 1️⃣ Import Libraries and Configuration

**Task**: Import all necessary libraries and set up configuration parameters.

**Requirements**:
- Import PyTorch, torchvision, and GAN-specific libraries
- Import visualization and utility libraries
- Set random seeds for reproducibility
- Configure training hyperparameters

In [ ]:
# TODO: Import core PyTorch libraries
# torch, torch.nn, torch.optim
# torchvision.transforms, torchvision.utils

# TODO: Import dataset and data handling
# torch.utils.data (DataLoader, Dataset)
# kagglehub for dataset download

# TODO: Import visualization and utilities
# matplotlib.pyplot, numpy, PIL.Image
# itertools, tqdm, random, os

# TODO: Set random seeds for reproducibility (use seed=42)
# torch.manual_seed, np.random.seed, random.seed

# TODO: Check device availability and print

# TODO: Define configuration parameters:
IMG_SIZE = 64           # Image resolution (64x64)
BATCH_SIZE = 4          # Batch size (small due to memory constraints)
LEARNING_RATE = 0.0002  # Learning rate for both generators and discriminators
BETA1 = 0.5             # Beta1 for Adam optimizer
BETA2 = 0.999           # Beta2 for Adam optimizer
NUM_EPOCHS = 15         # Number of training epochs
LAMBDA_CYCLE = 10.0     # Weight for cycle consistency loss
LAMBDA_IDENTITY = 5.0   # Weight for identity loss
N_RESIDUAL_BLOCKS = 6   # Number of residual blocks in generator

## 2️⃣ Dataset Download and Loading

**Task**: Download the Selfie2Anime dataset and create data loaders.

**Requirements**:
- Download dataset using kagglehub
- Create custom dataset class for unpaired data
- Apply appropriate transformations (resize, normalize)
- Create train and test data loaders

In [ ]:
# TODO: Download the Selfie2Anime dataset
# path = kagglehub.dataset_download("arnaud58/selfie2anime")
# print("Path to dataset files:", path)

# TODO: Define image transformations:
# Use transforms.Compose with:
# - Resize to (IMG_SIZE, IMG_SIZE)
# - ToTensor()
# - Normalize with mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5) for [-1,1] range

# TODO: Set up dataset paths:
# train_selfie_path = os.path.join(dataset_path, "trainA")  # Selfie images
# train_anime_path = os.path.join(dataset_path, "trainB")   # Anime images
# test_selfie_path = os.path.join(dataset_path, "testA")    # Test selfie images
# test_anime_path = os.path.join(dataset_path, "testB")     # Test anime images

# TODO: Create custom dataset class (SelfieAnimeDataset)
# Requirements:
# - __init__(self, root_selfie, root_anime, transform=None)
# - Load image file lists from both domains
# - __len__ returns max length of both domains
# - __getitem__ returns (selfie_img, anime_img) using modulo for cycling

# TODO: Create train and test datasets
# TODO: Create data loaders with appropriate batch size and shuffle
# TODO: Print dataset sizes and verify data loading

## 3️⃣ Generator Architecture

**Task**: Implement the Generator network with residual blocks.

**Requirements**:
- Create ResidualBlock class with skip connections
- Implement Generator with encoder-decoder structure
- Use reflection padding and instance normalization
- Include downsampling, residual blocks, and upsampling

In [ ]:
# TODO: Create ResidualBlock class:
# - Inherit from nn.Module
# - Use reflection padding, conv2d, instance norm, ReLU
# - Implement skip connection in forward pass: return x + conv_block(x)

# TODO: Create Generator class:
# In __init__:
# 1. Initial convolution block:
#    - ReflectionPad2d(3), Conv2d(3->64, kernel=7), InstanceNorm2d, ReLU
#
# 2. Downsampling layers (2 layers):
#    - Conv2d with stride=2, InstanceNorm2d, ReLU
#    - Features: 64 -> 128 -> 256
#
# 3. Residual blocks:
#    - Add N_RESIDUAL_BLOCKS ResidualBlock layers
#
# 4. Upsampling layers (2 layers):
#    - ConvTranspose2d with stride=2, InstanceNorm2d, ReLU
#    - Features: 256 -> 128 -> 64
#
# 5. Output layer:
#    - ReflectionPad2d(3), Conv2d(64->3, kernel=7), Tanh()

# TODO: Implement forward method
# TODO: Test generator with random input to verify output shape

## 4️⃣ Discriminator Architecture

**Task**: Implement the Discriminator network for adversarial training.

**Requirements**:
- Create PatchGAN discriminator architecture
- Use leaky ReLU activations and instance normalization
- Output patch-based predictions rather than single value
- Handle both real and fake image discrimination

In [ ]:
# TODO: Create Discriminator class:
# Create helper function discriminator_block(in_filters, out_filters, normalize=True):
# - Conv2d(kernel=4, stride=2, padding=1)
# - InstanceNorm2d (if normalize=True)
# - LeakyReLU(0.2)

# TODO: In __init__:
# Build discriminator using discriminator_block:
# - Layer 1: 3 -> 64 (no normalization)
# - Layer 2: 64 -> 128
# - Layer 3: 128 -> 256
# - Layer 4: 256 -> 512
# - Final: ZeroPad2d + Conv2d(512->1, kernel=4)

# TODO: Implement forward method
# TODO: Test discriminator with random input to verify output shape

## 5️⃣ Loss Functions

**Task**: Implement the three types of losses used in CycleGAN.

**Requirements**:
- Adversarial loss for generator and discriminator training
- Cycle consistency loss to ensure cycle A→B→A ≈ A
- Identity loss to preserve color composition
- Combine losses with appropriate weights

In [ ]:
# TODO: Create CycleGANLoss class:
# In __init__:
# - Store lambda_cyc and lambda_id weights
# - Initialize MSELoss for adversarial loss
# - Initialize L1Loss for cycle and identity losses

# TODO: Implement adversarial_loss(self, pred, target_is_real):
# - Create target tensor (ones for real, zeros for fake)
# - Return MSE loss between prediction and target

# TODO: Implement cycle_consistency_loss(self, real_images, cycled_images):
# - Return L1 loss between original and cycle-reconstructed images

# TODO: Implement identity_loss(self, real_images, same_images):
# - Return L1 loss between real images and same-domain generated images
# - Used when G_AB(B) should ≈ B and G_BA(A) should ≈ A

# TODO: Initialize loss function with configured weights

## 6️⃣ Model Initialization and Optimizers

**Task**: Initialize all models and optimizers for CycleGAN training.

**Requirements**:
- Create two generators: G_AB (Selfie→Anime) and G_BA (Anime→Selfie)
- Create two discriminators: D_A (for Selfie domain) and D_B (for Anime domain)
- Initialize optimizers for generators and discriminators separately
- Move all models to appropriate device

In [ ]:
# TODO: Initialize models and move to device:
# G_AB = Generator().to(device)  # Selfie to Anime
# G_BA = Generator().to(device)  # Anime to Selfie
# D_A = Discriminator().to(device)  # Discriminator for Selfie domain
# D_B = Discriminator().to(device)  # Discriminator for Anime domain

# TODO: Initialize optimizers:
# optimizer_G: Adam optimizer for both generators (itertools.chain)
# optimizer_D_A: Adam optimizer for discriminator A
# optimizer_D_B: Adam optimizer for discriminator B
# Use lr=LEARNING_RATE, betas=(BETA1, BETA2)

# TODO: Initialize loss function with configured weights
# TODO: Print model architectures and parameter counts

## 7️⃣ Training Function

**Task**: Implement the CycleGAN training loop for one epoch.

**Requirements**:
- Train generators with adversarial, cycle, and identity losses
- Train discriminators to distinguish real from fake images
- Alternate between generator and discriminator updates
- Track and return loss values for monitoring

In [ ]:
# TODO: Create train_epoch(epoch) function:

# TODO: Set all models to training mode
# TODO: Initialize running loss trackers

# TODO: For each batch (real_A, real_B):
#
# === TRAIN GENERATORS ===
# 1. Zero generator gradients
#
# 2. Identity Loss:
#    - loss_id_A = criterion.identity_loss(real_A, G_BA(real_A))
#    - loss_id_B = criterion.identity_loss(real_B, G_AB(real_B))
#    - loss_identity = (loss_id_A + loss_id_B) / 2
#
# 3. Adversarial Loss:
#    - fake_B = G_AB(real_A)
#    - loss_GAN_AB = criterion.adversarial_loss(D_B(fake_B), True)
#    - fake_A = G_BA(real_B)
#    - loss_GAN_BA = criterion.adversarial_loss(D_A(fake_A), True)
#    - loss_GAN = (loss_GAN_AB + loss_GAN_BA) / 2
#
# 4. Cycle Consistency Loss:
#    - recovered_A = G_BA(fake_B)
#    - loss_cycle_ABA = criterion.cycle_consistency_loss(real_A, recovered_A)
#    - recovered_B = G_AB(fake_A)
#    - loss_cycle_BAB = criterion.cycle_consistency_loss(real_B, recovered_B)
#    - loss_cycle = (loss_cycle_ABA + loss_cycle_BAB) / 2
#
# 5. Total Generator Loss:
#    - loss_G = loss_GAN + LAMBDA_CYCLE * loss_cycle + LAMBDA_IDENTITY * loss_identity
#    - loss_G.backward() and optimizer_G.step()
#
# === TRAIN DISCRIMINATOR A ===
# 1. Zero discriminator A gradients
# 2. Real loss: loss_real_A = criterion.adversarial_loss(D_A(real_A), True)
# 3. Fake loss: loss_fake_A = criterion.adversarial_loss(D_A(fake_A.detach()), False)
# 4. Total: loss_D_A = (loss_real_A + loss_fake_A) / 2
# 5. Backward and step
#
# === TRAIN DISCRIMINATOR B ===
# Similar to Discriminator A but for domain B
#
# TODO: Track running losses and return epoch averages

## 8️⃣ Training Loop and Monitoring

**Task**: Execute the full training process with progress monitoring.

**Requirements**:
- Train for specified number of epochs
- Display loss values and training progress
- Save sample images during training for visual monitoring
- Track loss curves for analysis

In [ ]:
# TODO: Initialize loss tracking lists
# g_losses = []
# d_losses = []

# TODO: Training loop:
# for epoch in range(NUM_EPOCHS):
#     g_loss, d_loss = train_epoch(epoch)
#     g_losses.append(g_loss)
#     d_losses.append(d_loss)
#
#     print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] - G_loss: {g_loss:.4f}, D_loss: {d_loss:.4f}")
#
#     # Save sample images every 5 epochs
#     if (epoch + 1) % 5 == 0:
#         TODO: Generate and save sample translations
#         - Set models to eval mode
#         - Get test batch
#         - Generate fake_B = G_AB(real_A) and fake_A = G_BA(real_B)
#         - Create comparison grid and save

# TODO: Plot training loss curves
# Create matplotlib plot showing generator and discriminator losses over epochs

## 9️⃣ Test Set Evaluation

**Task**: Evaluate the trained model on test data with comprehensive visualization.

**Requirements**:
- Generate 10 selfie-to-anime translations
- Generate 10 anime-to-selfie translations  
- Display results in organized grid format
- Show original and generated images side by side

In [ ]:
# TODO: Create evaluate_on_test_set() function:
# 1. Set models to evaluation mode
# 2. Use torch.no_grad() for inference
# 3. Collect 10 test images from each domain
# 4. Generate translations using both generators
# 5. Create 4x10 subplot grid:
#    - Row 1: Original selfies
#    - Row 2: Generated anime (from selfies)
#    - Row 3: Original anime
#    - Row 4: Generated selfies (from anime)
# 6. Display with appropriate titles and no axes

# TODO: Call evaluation function and display results

## 🔟 Internet Images Evaluation

**Task**: Test the model on external images from the internet.

**Requirements**:
- Load 3 selfie images and 3 anime images from provided URLs
- Apply same preprocessing as training data
- Generate translations in both directions
- Display results to demonstrate generalization

In [ ]:
# TODO: Create load_internet_image_from_url(url) function:
# 1. Use requests to download image with appropriate headers
# 2. Handle errors and content type validation
# 3. Apply same transforms as training (resize, normalize)
# 4. Return processed tensor or None if failed

# TODO: Create evaluate_internet_images() function:
# 1. Define image URLs for selfies and anime
# Some images you can use, you can use your own.
# selfie_urls = [
#     'https://i.imgur.com/m7Em3S2.png',
#     'https://i.imgur.com/2lzy9Un.png',
#     'https://i.imgur.com/eg7lsZ2.png'
# ]
# anime_urls = [
#     'https://i.imgur.com/2lCFEIY.png',
#     'https://i.imgur.com/kYXGdqM.png',
#     'https://i.imgur.com/G6tQMt9.png'
# ]
#
# 2. Load images and generate translations
# 3. Create 2x6 subplot showing:
#    - Row 1: Internet selfie, Generated anime (repeat for 3 pairs)
#    - Row 2: Internet anime, Generated selfie (repeat for 3 pairs)

# TODO: Call evaluation function

## 1️⃣1️⃣ Model Saving and Analysis

**Task**: Save trained models and analyze the learning process.

**Requirements**:
- Save all trained model state dictionaries
- Analyze training stability and convergence
- Discuss quality of generated images
- Document observations and potential improvements

In [ ]:
# TODO: Save all trained models:
# torch.save(G_AB.state_dict(), 'generator_AB.pth')
# torch.save(G_BA.state_dict(), 'generator_BA.pth')
# torch.save(D_A.state_dict(), 'discriminator_A.pth')
# torch.save(D_B.state_dict(), 'discriminator_B.pth')

# TODO: Analyze training results:
# 1. Comment on loss convergence and stability
# 2. Evaluate quality of selfie-to-anime translations
# 3. Evaluate quality of anime-to-selfie translations
# 4. Discuss which direction works better and why
# 5. Identify potential improvements (longer training, different architectures, etc.)

print("Training completed and models saved!")
print("Analysis: [Write your observations about the training process and results]")

## 📝 Evaluation Criteria

Your homework will be evaluated based on:

1. **Implementation Correctness (40%)**
   - Proper Generator and Discriminator architectures
   - Correct loss function implementations (adversarial, cycle, identity)
   - Working training loop with proper gradient updates
   - Appropriate data loading and preprocessing

2. **Model Performance (30%)**
   - Model trains without errors for full epoch count
   - Generated images show clear style transfer
   - Loss curves demonstrate learning progress
   - Reasonable visual quality in translations

3. **Code Quality (30%)**
   - Clean, readable code with proper comments
   - Efficient tensor operations and device management
   - Proper use of PyTorch best practices
   - Well-structured class definitions